<a href="https://colab.research.google.com/github/Caedessan/kaggle_toxic_comment/blob/main/NLP_TOXIC_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_train = train["comment_text"]
list_sentences_test = test["comment_text"]

In [ ]:
max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

In [ ]:
maxlen = 200
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [ ]:
inp = Input(shape=(maxlen, ))
embed_size = 128
x = Embedding(max_features, embed_size)(inp)
x = LSTM(60, return_sequences=True,name='lstm_layer')(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.1)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)

In [ ]:
model = Model(inputs=inp, outputs=x)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 200)]             0         
                                                                 
 embedding_2 (Embedding)     (None, 200, 128)          2560000   
                                                                 
 lstm_layer (LSTM)           (None, 200, 60)           45360     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 60)               0         
 balMaxPooling1D)                                                
                                                                 
 dropout_1 (Dropout)         (None, 60)                0         
                                                                 
 dense (Dense)               (None, 50)                3050      
                                                             

In [ ]:
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [ ]:
model.fit(X_t,y,
          batch_size=32,
          epochs=2,
          validation_split=0.1)

Epoch 1/2
4488/4488 [==============================] - 757s 168ms/step - loss: 0.0712 - accuracy: 0.9541 - val_loss: 0.0492 - val_accuracy: 0.9940
Epoch 2/2
4488/4488 [==============================] - 755s 168ms/step - loss: 0.0450 - accuracy: 0.9864 - val_loss: 0.0475 - val_accuracy: 0.9940


In [ ]:
 output = model.predict(X_te)

array([[9.9648350e-01, 4.0517703e-01, 9.7257590e-01, 6.6189677e-02,
        8.9467919e-01, 1.6294062e-01],
       [1.1080205e-03, 9.5337418e-07, 1.2961030e-04, 1.0236971e-05,
        1.1986456e-04, 1.7015949e-05],
       [3.3254027e-03, 7.6660917e-06, 4.6887994e-04, 6.4924949e-05,
        5.1563978e-04, 6.9484580e-05],
       ...,
       [1.3003051e-03, 7.2703682e-07, 1.2993813e-04, 5.0107446e-06,
        1.3449788e-04, 3.0970696e-05],
       [1.6446233e-02, 1.6688995e-05, 1.4036298e-03, 1.1306677e-04,
        2.5397837e-03, 9.0080500e-04],
       [9.6874493e-01, 1.2072086e-02, 6.8970990e-01, 2.4845302e-03,
        5.3435683e-01, 7.1528852e-03]], dtype=float32)

In [ ]:
output.to_csv('submission.csv', index=False)